# Welcome to the Brown University Datathon 2024!

Welcome to notebook 0! This notebook highlights the learning objectives for the datathon and prepares the datasets for the subsequent notebooks.

Happy coding!!

## Objectives and Tracks

The objective is to investigate the impact of the data issues that exist in electronic health records on downstream clinical prediction tasks. We shall investigate the effect of a faulty pulse oximeter reading, the effect of a missing serum lactate level, and the effect of the combination of the two on mortality prediction in the hospital. We will be creating 3 "altered" datasets in addition to the original WiDS dataset:

1. A dataset where the SpO2 of the Black patients will be increased by 10%

2. A dataset where we drop the serum lactate measurements of Black patients

3. A dataset where the SpO2 of the Black patients will be increased by 10% and their serum lactate is dropped


We exaggerate these data issues to get a sense of their impact on machine learning which surprisingly has not been sufficiently explored by the machine learning community.



## Schedule

* **First hour:** data visualization and table one of the WiDS dataset. (**Notebook 1**)
  
* **Second hour:** Build a mortality prediction model using the [WiDS dataset](https://physionet.org/content/widsdatathon2020/1.0.0/). Evaluate performance across race-ethnicities in the test set. (**Notebook 2**)
  
* **Third hour:** Build a mortality prediction model using one of three altered datasets. Use the same test set as above, but with the new features. (**Notebook 3**)

* **Fourth hour:** Compare the two models and prepare presentation for Day 2.

## Materials (online)

* [WiDS dataset](https://physionet.org/content/widsdatathon2020/1.0.0) - please download the data ("training_v2.csv") from here, and run this notebook to create the train and test subsets with the modified features - **before the datathon!!** 

* [Data Dictionary](https://physionet.org/content/widsdatathon2020/1.0.0/data/WiDS_Datathon_2020_Dictionary.csv) - to understand what the variables mean

* [Datathon GitHub](https://github.com/joamats/mit-brown-datathon) - to move onto the next notebooks!


## Dataset Preparation

### Import Libraries

In [366]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

### Define your working directory

In [367]:
os.chdir("/Users/joaomatos/Documents/mit-brown-datathon")

### Load the data

In [368]:
data = pd.read_csv("data/training_v2.csv")

In [369]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
    display(data.head())

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,1

### SpO2 modifications (1)

#### baseline distributions

In [370]:
data['d1_spo2_min'].isna().mean()

0.0036308920218507735

In [371]:
data['d1_spo2_min'].describe()

count    91380.000000
mean        90.454826
std         10.030069
min          0.000000
25%         89.000000
50%         92.000000
75%         95.000000
max        100.000000
Name: d1_spo2_min, dtype: float64

In [372]:
data['d1_spo2_max'].describe()

count    91380.000000
mean        99.241836
std          1.794181
min          0.000000
25%         99.000000
50%        100.000000
75%        100.000000
max        100.000000
Name: d1_spo2_max, dtype: float64

#### add bias to Black patient's SpO2

In [373]:
delta_to_add = 10

data['d1_spo2_min_new'] = data.apply(
    lambda row: 
    row.d1_spo2_min + delta_to_add if 
        ((row.d1_spo2_min + delta_to_add) <= 100) & (row.ethnicity == 'African American')
    else (100 if 
        ((row.d1_spo2_min + delta_to_add) > 100) & (row.ethnicity == 'African American')
    else (row.d1_spo2_min)),
    axis=1
)

#### compare both

before

In [374]:
data.loc[data.ethnicity == 'African American','d1_spo2_min'].describe()

count    9501.000000
mean       91.052837
std        11.702494
min         0.000000
25%        90.000000
50%        94.000000
75%        97.000000
max       100.000000
Name: d1_spo2_min, dtype: float64

after

In [375]:
data.loc[data.ethnicity == 'African American','d1_spo2_min_new'].describe()

count    9501.000000
mean       97.048311
std        10.129773
min        10.000000
25%       100.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: d1_spo2_min_new, dtype: float64

there is a 3% difference in the median!

### Lactate modifications (2)

#### baseline missingness

In [376]:
data['d1_lactate_max'].notnull().mean()

0.2542387665870705

In [377]:
data.loc[data.ethnicity == 'African American', 'd1_lactate_max'].isna().mean()

0.7387661045354562

In [378]:
data.loc[data.ethnicity == 'Caucasian', 'd1_lactate_max'].isna().mean()

0.7428272310565333

#### drop all the lactate values for Black patients

In [379]:

data['d1_lactate_max_new'] = data.apply(
    lambda row: 
    np.nan if 
        row.ethnicity == 'African American'
    else row.d1_lactate_max,
    axis=1
)

#### new missingness

In [380]:
data.loc[data.ethnicity == 'African American', 'd1_lactate_max_new'].isna().mean()

1.0

### Limit columns that we share
This is being done to ensure that respiratory function is not represented in other variables, reducing the impact of the extra bias that we are creating

By keeping a limited set of features, we can guarantee that the bias will have some impact

In [381]:
data = data[[
     # ids
     'encounter_id',
     'patient_id',
     'hospital_id',
     # patient
     'age',
     'ethnicity',
     'gender',
     'bmi',
     # icu stay info
     'icu_admit_source',
     'icu_type',
     # vital signs
     'd1_heartrate_max',
     'd1_heartrate_min',
     'd1_mbp_max',
     'd1_mbp_min',
     'd1_sysbp_max',
     'd1_sysbp_min',
     'd1_diasbp_max',
     'd1_diasbp_min',
     'd1_resprate_max',
     'd1_resprate_min',
     'd1_temp_max',
     'd1_temp_min',
     # labs
     'd1_albumin_min',
     'd1_bilirubin_max',
     'd1_bun_max',
     'd1_calcium_max',
     'd1_calcium_min',
     'd1_creatinine_max',
     'd1_glucose_max',
     'd1_glucose_min',
     'd1_hco3_min',
     'd1_hemaglobin_min',
     'd1_hematocrit_min',
     'd1_inr_max',
     'd1_platelets_min',
     'd1_potassium_max',
     'd1_potassium_min',
     'd1_sodium_max',
     'd1_sodium_min',
     'd1_wbc_max',
     # target
     'hospital_death',
     # study features
     'd1_spo2_min_new',
     'd1_lactate_max_new',
     'd1_spo2_min',
     'd1_lactate_max',

]]


### Train and test split

#### 80-20% split

In [382]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

In [383]:
data_train.shape

(73370, 44)

In [384]:
data_test.shape

(18343, 44)

#### check balancing of the mortality outcome

In [385]:
data.hospital_death.mean()

0.08630183289173836

In [386]:
data_train.hospital_death.mean()

0.08624778519830993

In [387]:
data_test.hospital_death.mean()

0.08651801777244726

not too different, we're good to go!

#### save the dataframes as CSV files for the next notebooks!

create a subfolder called 'data' within our directory

In [388]:
if not os.path.exists('data_split'):
    os.makedirs('data_split')

save both dataframes

In [389]:
data_train.to_csv('data_split/wids_train.csv')

In [390]:
data_test.to_csv('data_split/wids_test.csv')